<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px"> 
<br>
<b>Phase Curve of Solar System Objects</b> <br>
Contact authors: Christina Williams and Yumi Choi<br>
Last verified to run: <i>2023-07-dd</i> <br>
LSST Science Piplines version: Weekly <i>2023_21</i> <br>
Container Size: medium <br>
Targeted learning level: intermediate <br>

**Description:** Investigate the derivation of phase curves for DP0.3 solar system objects and explore DP0.3-provided phase curve parameters. 

**Skills:** Use various TAP tables, including joining multiple tables. Derive and compare phase curves. Explore accuracy of the phase curve parameters as a function of obervations.

**LSST Data Products:** TAP tables dp03_catalogs_10yr.SSObject, dp03_catalogs_10yr_10yr.MPCORB, dp03_catalogs_10yr_10yr.DiaSource, dp03_catalogs_10yr_10yr.SSSource

**Packages:** numpy, scipy, pandas, matplotlib, seaborn, lsst.rsp

**Credit:**
Inspired by a jupyter notebook developed by Queen's University Belfast Planet Lab (including Brian Rogers, Niall McElroy, and Meg Schwamb). Standalone functions for phase curve fitting were developed by Pedro Bernardinelli. References: <a href="https://ui.adsabs.harvard.edu/abs/2010Icar..209..542M/abstract">Muinonen et al. (2010)</a> and <a href="http://astronotes.co.uk/blog/2018/05/28/determining-the-h-g-parameters-of-atlas-asteroid-phase-curves.html">David Young's webpage.</a> Please consider acknowledging them if this notebook is used for the preparation of journal articles, software releases, or other notebooks.Please consider acknowledging them if this notebook is used for the preparation of journal articles, software releases, or other notebooks.

**Get Support:** Find DP0.3-related documentation and resources at <a href="https://dp0-3.lsst.io">dp0-3.lsst.io</a>. Questions are welcome as new topics in the <a href="https://community.lsst.org/c/support/dp0">Support - Data Preview 0 Category</a> of the Rubin Community Forum. Rubin staff will respond to all questions posted there.

In [ ]:
%load_ext pycodestyle_magic
%flake8_on
import logging
logging.getLogger("flake8").setLevel(logging.FATAL)

## 1. Introduction

This notebook will explore the properties of solar system objects in the DP0.3 dataset by using the science example of measuring their phase curves. 

TBD add narrative introduction. Define reduced magnitudes, absolute magnitudes, phase curves, distance types, etc. Say why phase curve fitting is important.narrative introduction. Define reduced magnitudes, absolute magnitudes, phase curves, distance types, etc. Say why phase curve fitting is important.

The DP0.3 catalog contains both real and simulated Solar System objects (including asteroids, near-earth objects, Trojans, trans-Neptunian objects) In the real survey, these objects will change position between each Rubin image. The DP0.3 catalog simulates the measurements of object movements between images, and these can be used to estimate their intrinsic properties, orbital parameters, etc. An important way to characterize intrinsic properties of solar system objects is by measuring its "phase curve", which is the object brightness as a function of its "solar phase angle" (the angle made between the line of sight from the object to the sun, and the line of sight from the object to earth; see diagram at <a href="http://astronotes.co.uk/blog/2018/05/28/determining-the-h-g-parameters-of-atlas-asteroid-phase-curves.html">David Young's webpage.</a>)


In order to reveal the intrinsic properties of the astroid (such as its surface properties and albedo; and as a result helps determine its class of solar system body) we first must turn apparent magnitudes as a function of time (what is measured by LSSTd ata) into "reduced magnitude", which takes into account the relative distances between the asteroid and the sun/earth (heliocentric/topocentric distances) at each observation. Thus, reduced magnitude is normalized such that it is the brightness of an asteroid as if it is observed at 1 AU from both the Sun and the Earth. 

In this notebook we will show an example of measuring the phase curve of solar system bodies using the DP0.3 simulated catalogs. In Section 2 we will manually perform the phase curve fitting on the DP0.3 data on one solar system object, using 3 different parametrizations of a phase curve model. In Section 3, we will compare that to the automated phase curve fitting that is performed as part of the LSST data products, and available in the SSObject Table (do we not do this anymore?). Lastly in Section 4 we will aggregate the phase curve fits for a number of solar system bodies in DP0.3 and study how the quality of the fit depends on LSST observations (which additionally provides some insight into expectations for real LSST data).



### 1.1 Package Imports

The [matplotlib](https://matplotlib.org/) (and especially sublibrary `matplotlib.pyplot`), [numpy](http://www.numpy.org/), and [scipy](https://scipy.org/) libraries are widely used Python libraries for plotting and scientific computing, and model fitting.

The `lsst.rsp` package provides access to the Table Access Protocol (TAP) service for queries to the DP0 catalogs.

The [seaborn](https://seaborn.pydata.org/) package provides statistical data visualization with aesthetic and informative graphics.

The [pandas](https://pandas.pydata.org/) package enables table and dataset manipulation (TBD are we using this)

Finally we import `warnings` to do what?


In [ ]:
# general python packages
import numpy as np
from scipy.interpolate import CubicSpline
from scipy.optimize import leastsq
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

# LSST package for TAP queries
from lsst.rsp import get_tap_service

### 1.2 Define Functions and Parameters

# 1.2.1 Set up some plotting defaults:

In [ ]:
plt.style.use('tableau-colorblind10')
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']
params = {'axes.labelsize': 15,
          'font.size': 15,
          'legend.fontsize': 12}
plt.rcParams.update(params)

To supress NumPy RuntimeWarning

In [ ]:
np.seterr(divide='ignore')

Define functions for phase curve fitting:

In [ ]:
# Constants values taken from sbpy (https://sbpy.org) for convenience
A = [3.332, 1.862]
B = [0.631, 1.218]
C = [0.986, 0.238]

alpha_12 = np.deg2rad([7.5, 30., 60, 90, 120, 150])

phi_1_sp = [7.5e-1, 3.3486016e-1, 1.3410560e-1, 5.1104756e-2, 2.1465687e-2,
            3.6396989e-3]
phi_1_derivs = [-1.9098593, -9.1328612e-2]

phi_2_sp = [9.25e-1, 6.2884169e-1, 3.1755495e-1, 1.2716367e-1, 2.2373903e-2, 
            1.6505689e-4]
phi_2_derivs = [-5.7295780e-1, -8.6573138e-8]

alpha_3 = np.deg2rad([0.0, 0.3, 1., 2., 4., 8., 12., 20., 30.])

phi_3_sp = [1., 8.3381185e-1, 5.7735424e-1, 4.2144772e-1, 2.3174230e-1, 
            1.0348178e-1, 6.1733473e-2, 1.6107006e-2, 0.]
phi_3_derivs = [-1.0630097, 0]

phi_1 = CubicSpline(alpha_12, phi_1_sp, bc_type=((1,phi_1_derivs[0]),(1,phi_1_derivs[1])))
phi_2 = CubicSpline(alpha_12, phi_2_sp, bc_type=((1,phi_2_derivs[0]),(1,phi_2_derivs[1])))
phi_3 = CubicSpline(alpha_3, phi_3_sp, bc_type=((1,phi_3_derivs[0]),(1,phi_3_derivs[1])))

In [ ]:
def HG_model(phase, params):
    """
    Compute HG model phase curve for a given set
    of parameters. The simplest 2-parameter model.

    Parameters
    ----------
    phase: ndarray
        phase angle in radians
    params: list
        phase curve parameters

    Returns
    -------
    computed reduced magnitude: ndarray
    """
    
    sin_a = np.sin(phase)
    tan_ah = np.tan(phase/2)
    
    W = np.exp(-90.56 * tan_ah * tan_ah)    
    scale_sina = sin_a/(0.119 + 1.341*sin_a - 0.754*sin_a*sin_a)
    
    phi_1_S = 1 - C[0] * scale_sina
    phi_2_S = 1 - C[1] * scale_sina
    
    phi_1_L = np.exp(-A[0] * np.power(tan_ah, B[0]))
    phi_2_L = np.exp(-A[1] * np.power(tan_ah, B[1]))
    
    phi_1 = W * phi_1_S + (1-W) * phi_1_L
    phi_2 = W * phi_2_S + (1-W) * phi_2_L
    
    return params[0] - 2.5*np.log10((1-params[1])* phi_1 + (params[1]) * phi_2) 

In [ ]:
def HG1G2_model(phase, params):
    """
    Compute HG1G2 model phase curve for a given set 
    of parameters. This is a 3-parameter model, which works best
    when sufficiently long phaseangle coverage is available.

    Parameters
    ----------
    phase: ndarray
        phase angle in radians
    params: list
        phase curve parameters

    Returns
    -------
    computed reduced magnitude: ndarray     
    """

    tan_ah = np.tan(phase/2)

    phi_1_ev = phi_1(phase)  
    phi_2_ev = phi_2(phase)  
    phi_3_ev = phi_3(phase)  

    msk = phase < 7.5 * np.pi/180

    phi_1_ev[msk] = 1-6*phase[msk]/np.pi 
    phi_2_ev[msk] = 1- 9 * phase[msk]/(5*np.pi)

    phi_3_ev[phase > np.pi/6] = 0

    return params[0] - 2.5 * np.log10(params[1] * phi_1_ev + params[2] * phi_2_ev + (1-params[1]-params[2]) * phi_3_ev)

In [ ]:
def HG12_model(phase, params): 
    """
    Compute HG12 model phase curve for a given set 
    of parameters. This is a 2-parameter, simplified version
    of HG1G2 model, which is useful when phaseangle coverage is shorter. 

    Parameters
    ----------
    phase: ndarray
        phase angle in radians
    params: list
        phase curve parameters

    Returns
    -------
    computed reduced magnitude: ndarray
    """
    
    if params[1] >= 0.2:
        G1 = +0.9529*params[1] + 0.02162 
        G2 = -0.6125*params[1] + 0.5572
    else:
        G1 = +0.7527*params[1] + 0.06164  
        G2 = -0.9612*params[1] + 0.6270

    return HG1G2_model(phase, [params[0], G1, G2])

In [ ]:
def wd(params, mag, phase, mag_err, model):
    """
    Compute weighted deviation for a given model.
    """
    
    pred = model(phase, params)
    
    return (mag - pred)/mag_err

In [ ]:
def fit(mag, phase, sigma, model=HG12_model, params=[0.1]):
    """
    Fit phase curve for given observations to a designated model.
    
    Parameters
    ----------
    mag: ndarray
        reduced magnitude
    phase: ndarray
        phase angle in degrees
    sigma: ndarray
        uncertainty in magnitude
    model: function (default=HG12_model)
        phase curve model function
    params: list (default=[0.1])
        phase curve paramters

    Returns
    -------
    sol: tuple
        best-fit solution
    """
    
    phase = np.deg2rad(phase)
    sol = leastsq(wd, [mag[0]] + params, (mag, phase, sigma, model), full_output = True)

    return sol

In [ ]:
def fit_models(reducedMag, magSigma, phaseAngle, verbose=False):
    """
    Fit phase curves for given observations to 3 different models.
    
    Parameters
    ----------
    reducedMag: ndarray
        reduced magnitude
    magSigma: ndarray
        uncertainty in magnitude
    phaseAngle: ndarray
        phase angle in degrees

    Returns
    -------
    solutions: dict
        Best-fit solutions for each model
    """

    # We fit observations using each one of the HG, HG12 and HG1G2 models 
    # and store the resulting solutions in a dictionary of dictionaries.
    # Save np.nan values when the fit has not been converged
    solutions = {}

    # Let's do HG first
    sol_HG = fit(reducedMag, phaseAngle, magSigma, model=HG_model)

    solutions['HG'] = {}  
    try:
        solutions['HG']['chi2'] = np.sum(sol_HG[2]['fvec']**2)
        solutions['HG']['H'] = sol_HG[0][0]
        solutions['HG']['G'] = sol_HG[0][1]
        solutions['HG']['H_err'] = np.sqrt(sol_HG[1][0,0])
        solutions['HG']['G_err'] = np.sqrt(sol_HG[1][1,1])
        solutions['HG']['cov'] = sol_HG[1]
    except TypeError:
        if verbose:
            print('HG model is not converging')
        solutions['HG']['chi2'] = np.nan
        solutions['HG']['H'] = np.nan
        solutions['HG']['G'] = np.nan
        solutions['HG']['H_err'] = np.nan
        solutions['HG']['G_err'] = np.nan
        solutions['HG']['cov'] = np.nan            

    # Now HG12
    sol_HG12 = fit(reducedMag, phaseAngle, magSigma, model=HG12_model)

    solutions['HG12'] = {}
    try:
        solutions['HG12']['chi2'] = np.sum(sol_HG12[2]['fvec']**2)
        solutions['HG12']['H'] = sol_HG12[0][0]
        solutions['HG12']['G12'] = sol_HG12[0][1]
        solutions['HG12']['H_err'] = np.sqrt(sol_HG12[1][0,0])
        solutions['HG12']['G12_err'] = np.sqrt(sol_HG12[1][1,1])
        solutions['HG12']['cov'] = sol_HG12[1]
    except TypeError:
        if verbose:
            print('HG12 model is not converging')
        solutions['HG12']['chi2'] = np.nan
        solutions['HG12']['H'] = np.nan
        solutions['HG12']['G12'] = np.nan
        solutions['HG12']['H_err'] = np.nan
        solutions['HG12']['G12_err'] = np.nan
        solutions['HG12']['cov'] = np.nan

    # Finally, HG1G2 - note this returns an extra parameter
    sol_HG1G2 = fit(reducedMag, phaseAngle, magSigma, model=HG1G2_model, params=[0.1, 0.1])
    
    solutions['HG1G2'] = {}
    try:
        solutions['HG1G2']['chi2'] = np.sum(sol_HG1G2[2]['fvec']**2)
        solutions['HG1G2']['H'] = sol_HG1G2[0][0]
        solutions['HG1G2']['G1'] = sol_HG1G2[0][1]
        solutions['HG1G2']['G2'] = sol_HG1G2[0][2]
        solutions['HG1G2']['H_err'] = np.sqrt(sol_HG1G2[1][0,0])
        solutions['HG1G2']['G1_err'] = np.sqrt(sol_HG1G2[1][1,1])
        solutions['HG1G2']['G2_err'] = np.sqrt(sol_HG1G2[1][2,2])
        solutions['HG1G2']['cov'] = sol_HG1G2[1]
    except TypeError:
        if verbose: 
            print('HG1G2 model is not converging')
        solutions['HG1G2']['chi2'] = np.nan
        solutions['HG1G2']['H'] = np.nan
        solutions['HG1G2']['G1'] = np.nan
        solutions['HG1G2']['G2'] = np.nan
        solutions['HG1G2']['H_err'] = np.nan
        solutions['HG1G2']['G1_err'] = np.nan
        solutions['HG1G2']['G2_err'] = np.nan
        solutions['HG1G2']['cov'] = np.nan
        
    return solutions

## 2. Querying the DP0.3 tables and fitting phase curves of SS objects

### 2.1 Create the Rubin TAP Service Client

Get an instance of the TAP service, and assert that it exists.

In [ ]:
service = get_tap_service("ssotap")
assert service is not None

### 2.2 Querying the DP0.3 SSObject and MPCORB catalogs

For phase curve fitting, we need apparent magnitudes & uncertainties, phase angles, topocentric (tdist) and heliocentric (rdist) distances.

_In the ssObjectTable, the column G12 is correct in that the final table
should contain the parameter G12, however, the existing version of the column
actually contains parameter G (and therefore should be compared to the HG_model
not HG12 model. The plan is to replace the contents of G12 column with actual G12
fit parameters in the final version. This needs to be checked before release._ 

To keep a reasonable run time for this tutorial, we limit our query, and select sources with number of objects in the ssObject table (numObs) > 3000 and arc of LSST observations (arc) > 3000 days. 




In [ ]:
nobs_thrh = '3000' # Number of LSST observations
arc_thrh = '3000' # Arc of LSST observations; units in days

In [ ]:
# Getting a table of columns for unique objects with number of observations > nobs_thrh and arc longer than arc_thrh
#    sso.uH, sso.uHerr, sso.uG12, sso.uG12err, sso.uH_uG12_Cov,
#    sso.yH, sso.yHerr, sso.yG12, sso.yG12err, sso.yH_yG12_Cov

query = """
SELECT 
    mpc.ssObjectId, mpc.q, mpc.mpcG, mpc.mpcH,
    sso.arc, sso.numObs, 
    sso.g_H, sso.g_Herr, sso.g_G12, sso.g_G12err, sso.g_H_gG12_Cov,
    sso.r_H, sso.r_Herr, sso.r_G12, sso.r_G12err, sso.r_H_rG12_Cov,
    sso.i_H, sso.i_Herr, sso.i_G12, sso.i_G12err, sso.i_H_iG12_Cov,
    sso.z_H, sso.z_Herr, sso.z_G12, sso.z_G12err, sso.z_H_zG12_Cov
FROM 
    dp03_catalogs_10yr.MPCORB as mpc 
INNER JOIN dp03_catalogs_10yr.SSObject as sso 
ON mpc.ssObjectId = sso.ssObjectId 
WHERE sso.numObs > {} AND sso.arc > {} ORDER by sso.ssObjectId 
""".format(nobs_thrh, arc_thrh)

df_uniqueObj = service.search(query).to_table()
df_uniqueObj

### 2.3 Querying the DP0.3 DiaSource and SSSource catalogs

Getting time series data for the unique objects selected above. This query usually takes ~30 sec.

In [ ]:
#    dia.magSigma is now magErr
# dia.filter now dia.band
query = """
SELECT 
    dia.ssObjectId, dia.diaSourceId, dia.mag,
    dia.magErr, dia.band, dia.midPointMjdTai, 
    sss.phaseAngle, sss.topocentricDist, sss.heliocentricDist
FROM 
    dp03_catalogs_10yr.DiaSource as dia
INNER JOIN 
    dp03_catalogs_10yr.SSSource as sss
ON 
    dia.diaSourceId = sss.diaSourceId
WHERE 
    dia.ssObjectId 
    IN (
        SELECT sso.ssObjectId
        FROM dp03_catalogs_10yr.SSObject as sso 
        WHERE sso.numObs > {} AND sso.arc > {}
        )
ORDER by dia.ssObjectId 
""".format(nobs_thrh, arc_thrh)

df_indivObs = service.search(query).to_table()
df_indivObs

Double check if the number of unique objects in df_indivObs is identical to that of df_uniqueObj.

In [ ]:
assert len(df_uniqueObj) == len(np.unique(df_indivObs['ssObjectId']))

### 2.4 Fitting phase curve per filter per unique object using three different fitting functions

Compute reduced magnitude for each observation and add it as a column to the df_indivObs. Reduced magnitude is normalized apparent magnitude of an asteroid to make as if it is observed at 1 AU from both the Sun and the Earth.

In [ ]:
dmag = -5. * np.log10(df_indivObs['topocentricDist']*df_indivObs['heliocentricDist'])
reduced_mag = df_indivObs['mag'] + dmag

df_indivObs.add_column(reduced_mag, name='reducedMag')

Fitting phase curve per filter per unique object using three different fitting functions. This cell takes ~6 min for 269 unique objects with the medium container size.

In [ ]:
fitted_array = []

for iobj in df_uniqueObj['ssObjectId']:
    idx = df_indivObs['ssObjectId'] == iobj
    df_tmp = df_indivObs[idx]
    filts = np.unique(df_tmp['band'])
    for ifilt in filts:
        idx_filt = df_tmp['band'] == ifilt
        nobs_ifilt = len(df_tmp[idx_filt])
    
        # number of observations needs to be greater than the number of fit parameter, which is 3.
        if nobs_ifilt > 3:
            x_fitted = fit_models(df_tmp['reducedMag'][idx_filt], 
                                  df_tmp['magErr'][idx_filt], 
                                  df_tmp['phaseAngle'][idx_filt]
                                  )
            fitted_array.append([iobj, ifilt, x_fitted])

In [ ]:
results = pd.DataFrame(fitted_array)
results.columns=['ssObjectId', 'fname', 'fit_param']

# Uncomment the line below if you want to display the contents of the model fit return:
# print(x_fitted)

Converting the fit parameter dictionary to individual columns in a pandas dataframe makes it easy to read each parameters. But it also takes some time. It might be great if we can come up with a more efficient way of storing and accessing all the necessary columns for making plots and doing analysis in later parts of this tutorial.

In [ ]:
results = results[['ssObjectId', 'fname']].join(pd.json_normalize(results.fit_param))

In [ ]:
# Plot the G vs H paramters from the two 2-parameter model fits: HG and HG12
fig = plt.figure(figsize=(13,8))
gs = fig.add_gridspec(2, 3, wspace=0, hspace=0)
axs = gs.subplots(sharex=True, sharey=True)
axs = axs.ravel()

for i,ifilt in enumerate(filts):
    axs[i].errorbar(results[results.fname==ifilt]['HG.H'],
                    results[results.fname==ifilt]['HG.G'], 
                    xerr=results[results.fname==ifilt]['HG.H_err'],
                    yerr=results[results.fname==ifilt]['HG.G_err'],
                    fmt='.', alpha=0.5, label='HG model')
    axs[i].errorbar(results[results.fname==ifilt]['HG12.H'],
                    results[results.fname==ifilt]['HG12.G12'], 
                    xerr=results[results.fname==ifilt]['HG12.H_err'],
                    yerr=results[results.fname==ifilt]['HG12.G12_err'], 
                    fmt='.', alpha=0.5, label='HG12 model')
        
    plt.yscale('log')
    axs[i].text(17, 7, ifilt+'-band')

fig.supxlabel('Absolute magnitude, H [mag]')
fig.supylabel('Slope parameter, G')
axs[0].legend(loc=2)

In [ ]:
# Plot example phase curves in u,g,r,i,z,y for a single object
iObj = 1 # you can pick any integer number between 0 and 268
sId = df_uniqueObj['ssObjectId'][iObj]
df_tmp = df_indivObs[df_indivObs['ssObjectId'] == sId]
phases = np.linspace(0,90,100)

for i,ifilt in enumerate(filts):
    idx = df_tmp['band'] == ifilt
    
    # Plot observations
    plt.errorbar(df_tmp['phaseAngle'][idx], df_tmp['reducedMag'][idx], yerr=df_tmp['magErr'][idx],
                 fmt='.', color=colors[i], alpha=0.5, label=ifilt)

    # Plot HG model
    HG_mag = HG_model(np.deg2rad(phases),
                     [results[(results.ssObjectId == sId) & (results.fname == ifilt)]['HG.H'].values,
                      results[(results.ssObjectId == sId) & (results.fname == ifilt)]['HG.G'].values])
    plt.plot(phases, HG_mag, color=colors[i], label ='HG 2-parameter model')
    
    #Plot HG12 model
    HG12_mag = HG12_model(np.deg2rad(phases),
                         [results[(results.ssObjectId == sId) & (results.fname == ifilt)]['HG12.H'].values,
                          results[(results.ssObjectId == sId) & (results.fname == ifilt)]['HG12.G12'].values])
    plt.plot(phases, HG12_mag, color=colors[i], linestyle='--',  label ='HG12 2-parameter model')
    
    # Plot HG1G2 model 
    HG1G2_mag = HG1G2_model(np.deg2rad(phases), 
                           [results[(results.ssObjectId == sId) & (results.fname == ifilt)]['HG1G2.H'].values,
                            results[(results.ssObjectId == sId) & (results.fname == ifilt)]['HG1G2.G1'].values,
                            results[(results.ssObjectId == sId) & (results.fname == ifilt)]['HG1G2.G2'].values])
    plt.plot(phases, HG1G2_mag, color=colors[i], linestyle='dotted',  label ='HG1G2 3-parameter model')
    
plt.xlim(df_tmp['phaseAngle'].min()-5, df_tmp['phaseAngle'].max()+5)
plt.ylim(df_tmp['reducedMag'].max()+0.5, df_tmp['reducedMag'].min()-0.5)
plt.xlabel('Phase Angle [deg]')
plt.ylabel('Reduced magnitude [mag]')
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left', ncol=2)
plt.title('ssObjectId = %d' % sId)

### 2.5 Explore phase curve fit uncertainty

In [ ]:
# Note: df_uniqueObj contains the model params and errors

# For the same unique object picked in the above cell, we delve into u- and i-filters 
# to demonstrate fit uncertainty in a sharp contrast

for ifilt in ['u', 'i']:
    idx = df_tmp['band'] == ifilt
    plt.errorbar(df_tmp['phaseAngle'][idx], df_tmp['reducedMag'][idx], yerr=df_tmp['magErr'][idx],
                 fmt='o', label=ifilt, zorder=10)

    # Grab the phase curve parameters for the HG12 model stored in SSObject Table.
    # In priciple, these parameters should be identical to those derived for the HG12 model above in this NB.
    HG_mag_sso = HG_model(np.deg2rad(phases), [df_uniqueObj[ifilt+'H'][iObj], df_uniqueObj[ifilt+'G12'][iObj]])
    plt.plot(phases, HG_mag_sso, 'k--', alpha=0.3)
    
    # Compute min/max values in reduced mag at each phase angle
    p1 = HG_model(np.deg2rad(phases), [df_uniqueObj[ifilt+'H'][iObj] + df_uniqueObj[ifilt+'Herr'][iObj], 
                                       df_uniqueObj[ifilt+'G12'][iObj] + df_uniqueObj[ifilt+'G12err'][iObj]])

    p2 = HG_model(np.deg2rad(phases), [df_uniqueObj[ifilt+'H'][iObj] - df_uniqueObj[ifilt+'Herr'][iObj], 
                                       df_uniqueObj[ifilt+'G12'][iObj] + df_uniqueObj[ifilt+'G12err'][iObj]])

    p3 = HG_model(np.deg2rad(phases), [df_uniqueObj[ifilt+'H'][iObj] + df_uniqueObj[ifilt+'Herr'][iObj], 
                                       df_uniqueObj[ifilt+'G12'][iObj] - df_uniqueObj[ifilt+'G12err'][iObj]])

    p4 = HG_model(np.deg2rad(phases), [df_uniqueObj[ifilt+'H'][iObj] - df_uniqueObj[ifilt+'Herr'][iObj], 
                                       df_uniqueObj[ifilt+'G12'][iObj] - df_uniqueObj[ifilt+'G12err'][iObj]])
    
    HG_magHigh = np.maximum(np.maximum(p1, p2), np.maximum(p3, p4))
    HG_magLow = np.minimum(np.minimum(p1, p2), np.minimum(p3, p4))
    
    plt.fill_between(phases, HG_magLow, HG_magHigh, alpha=0.3)

plt.xlim(df_tmp['phaseAngle'].min()-5, df_tmp['phaseAngle'].max()+5)
plt.ylim(df_tmp['reducedMag'].max()+0.5, df_tmp['reducedMag'].min()-0.5)
plt.xlabel('Phase Angle [deg]')
plt.ylabel('Reduced magnitude [mag]')
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.title('ssObjectId = %d' % sId)

Here we compare the typical apparent magnitude uncertainties per band to see how that impacts the fit. 

In [ ]:
# Compute meadian apparent magnitude and its median uncertainty in r-filter in this example
mag_med = []
magSigma_med = []
ifilt = 'r'
    
for iobj in df_uniqueObj['ssObjectId']:
    idx = df_indivObs['ssObjectId'] == iobj
    df_tmp = df_indivObs[idx]
    idx_filt = df_tmp['filter'] == ifilt
    
    mag_med.append(np.median(df_tmp['mag'][idx_filt]))
    magSigma_med.append(np.median(df_tmp['magSigma'][idx_filt]))
    
# Plot phase curve parameters vs. median apparent magnitude and its median uncertainty
plt.figure(figsize=(13,5))

plt.subplot(121)
plt.plot(mag_med, df_uniqueObj[ifilt+'Herr'], '.', alpha=.5, label=ifilt+'-band abs mag H')
plt.plot(mag_med, df_uniqueObj[ifilt+'G12err'], '.', alpha=.5, label=ifilt+'-band G parameter')
plt.ylabel('Uncertainty in phase curve fit')
plt.xlabel('Median apparent mag')
plt.legend()

plt.subplot(122)
plt.plot(magSigma_med, df_uniqueObj[ifilt+'Herr'],'.',alpha=.5)
plt.plot(magSigma_med, df_uniqueObj[ifilt+'G12err'],'.',alpha=.5)
plt.ylabel('Uncertainty in phase curve fit')
plt.xlabel('Median uncertainty in apparent mag')

plt.tight_layout()

Clean up all before moving on to population study

In [ ]:
del df_uniqueObj, df_indivObs, results

## 3. Population study of measurements of the SS objects DP0.3 tables 

Now that we have demonstrated how to fit a phase curve to DP0.3 data, experimented with 3 different models, and compared those to the automated fits that are stored in the SSObject Table, we look at the population in aggregate: population study with the fit results (e.g., how the fit values differ due to number of observations, or for near/far or bright/faint objects).

In [ ]:
# First, re-query for a table of columns for unique objects with number of observations and arc over larger
# dynamic range than before, and add some extra criteria of use in assessing uncertainty. To decide: is
# this second query is necessary, or should we adapt the earlier query and sub-sample it with 
# nobs_thrh and arc_thrh >3000, since the query takes a long time (~5 min).

nobs_thrh = 50 
arc_thrh = 100

query = """
SELECT TOP 50000
    mpc.ssObjectId, mpc.q, sso.arc, sso.numObs, 
    sso.uH, sso.uHerr, sso.uG12, sso.uG12err, 
    sso.gH, sso.gHerr, sso.gG12, sso.gG12err, 
    sso.rH, sso.rHerr, sso.rG12, sso.rG12err, 
    sso.iH, sso.iHerr, sso.iG12, sso.iG12err, 
    sso.zH, sso.zHerr, sso.zG12, sso.zG12err, 
    sso.yH, sso.yHerr, sso.yG12, sso.yG12err,
    sso.uNdata, sso.gNdata, sso.rNdata, 
    sso.iNdata, sso.zNdata, sso.yNdata
FROM 
    dp03_catalogs.MPCORB as mpc 
INNER JOIN dp03_catalogs.SSObject as sso 
ON mpc.ssObjectId = sso.ssObjectId 
WHERE sso.numObs > {} AND sso.arc > {} 
""".format(nobs_thrh, arc_thrh)

res = service.search(query).to_table().to_pandas()
res

### 3.1 Distribution of H and G parameters in all six filters


In [ ]:
# Plot the G vs H paramters from the two 2-parameter model fits: HG and HG12
fig = plt.figure(figsize=(13,8))
gs = fig.add_gridspec(2, 3, wspace=0, hspace=0)
axs = gs.subplots(sharex=True, sharey=True)
axs = axs.ravel()

for i,ifilt in enumerate(filts):
    sns.histplot(x=res[ifilt+'H'], y=res[ifilt+'G12'], bins=100, log_scale=(False, True), ax=axs[i])
    axs[i].text(20, 30, ifilt+'-band')
    axs[i].set(xlabel=None)
    axs[i].set(ylabel=None)

fig.supxlabel('Absolute magnitude, H [mag]')
fig.supylabel('Slope parameter, G')

### 3.2 Exploration based on numObs (total number) and arc (total baseline in days) of LSST observations

First, we have a look at how phase curve modeling accuracy might depend on "global" measurements (across all filters). For this section we compare total number of observations for a given source (numObs) and the total time baseline across which observations are made (arc of LSST observations) and explore whether or not these factors impact the quality of phase curve modeling. In general, the first plot shows that more observations yields lower uncertainties in the model fits. I have to admit I don't totally understand the second plot comparing uncertainties as a function of time baseline of observations (arc). I am guessing that longer arc does not translate directly to lower uncertainties, since most(?) of the objects in the catalog are distant (fewer sources close to sun?). Since distant objects can have quite long orbits, perhaps the requisite arc for a good fit varies substantially within the population? Maybe this plot would yield better results if the comparison is made after the sample is grouped by heliocentric distance or something.

In [ ]:
fig = plt.figure(figsize=(10,10))
gs = fig.add_gridspec(2, 2, wspace=0)
axs = gs.subplots(sharey=True)

# Uncertainty in i-band absolute magnitude vs. Number of LSST observations
sns.histplot(x=res['numObs'], y=res['iHerr'], bins=100, log_scale=(False, True), ax=axs[0,0])
axs[0,0].set_xlabel('Number of LSST observations')

# Uncertainty in i-band absolute magnitude vs. length of arc
sns.histplot(x=res['arc'], y=res['iHerr'], bins=100, ax=axs[0,1])
axs[0,1].set_xlabel('Arc [days]')

# Uncertainty in i-band slope parameter vs. Number of LSST observations
sns.histplot(x=res['numObs'], y=res['iG12err'], bins=100, ax=axs[1,0])
axs[1,0].set_xlabel('Number of LSST observations')

# Uncertainty in i-band slope parameter vs. length of arc
sns.histplot(x=res['arc'], y=res['iG12err'], bins=100, ax=axs[1,1])
axs[1,1].set_xlabel('Arc [days]')

### 3.3 Number of data used per band

The above plots compare numObs (total) with model fits (per band) which may not be the ideal metric since as one can see from the modeling of individual objects in different filters in Section 2.4, the quality of phase curves can vary quite a bit between filters. Instead, we can look at the number of datapoints included in the phase curve modeling on a per filter basis (i.e. rNdata for the r-band in the SSObject Table). Below, we look at the distribution of the number of observations in each filter used to model the phase curve per filter for all SSObjects which in total have more than 100 numObs and arc (as per query above). One can see that generally, r and i bands produce the most data points for recovering phase curves, while u-band produces the fewest. In the second plot, one can see that the 

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(13,4))

filters = ['u','g','r','i','z','y']
for i,ifilt in enumerate(filters):
    axs[0].hist(res[ifilt+'Ndata'], bins=100, range=(0,1300), histtype='step', label=ifilt+'-band')
    
axs[0].set_yscale('log')
axs[0].legend()
axs[0].set_xlabel('Number of data points per filter')
axs[0].set_ylabel('Number of sources')


sns.histplot(x=res['rNdata'], y=100*res['rHerr']/res['rH'], 
             bins=100, log_scale=(False, True), color='r', alpha=0.7, ax=axs[1])
sns.histplot(x=res['uNdata'], y=100*res['uHerr']/res['uH'], 
             bins=100, log_scale=(False, True), color='b', ax=axs[1])
    
axs[1].set_xlabel('Number of data points per filter')
axs[1].set_ylabel('Fractional Uncertainty in H [%]')
r_patch = mpatches.Patch(color='red', alpha=0.7, label='r-band')
u_patch = mpatches.Patch(color='blue', label='u-band')
axs[1].legend(handles=[r_patch, u_patch])

## Brainstorming additional parameters to use for characterizing the measurements 
These parameters could be useful for the end of section 3, where we explore what measurements we can use that are in df_indivObs and storing a "quality" parameter for them in a new column of df_uniqObj. 

Some ideas to compare uncertainties to (just pick a couple good informative ones): 
- number of observations (done)
- arc (done)
- sso.zNdata (or other filters) = number of datapoints used to fit phase curve (per filter; perhaps less noisy than total number of obs; done)
- Pick mean or median apparent mag photometric uncertainty for the best 68% of measurements? (done)
- mean or median reduced mag uncertainty for best 68% of measurements? (TBD)
- sso.MOID = minimum orbit intersection distance to earth (farther at fixed size means potentially it will stay faint always?) Pedro suggests this will not be a useful metric as it will change a lot and won't be included in 2nd DP0.3 release.
- Pick some median dia.snr = signal to noise ratio at which the source was detected in difference image (TBD)